Hier ist der erste Versuch vom Trimmed Mean, da wir so wenig Daten wie möglich verlieren wollen, haben wir uns für die Strategie entschieden, den Trimmed Mean für nur den Wetterdatensatz anzuwenden. Dabei haben wir den Trimmed Mean auf die 5 Städte angewendet und die kleinste und grösste Temperatur entfernt und von denn verbleibenden dreien den Mean genommen. Das Ziel ist es, denn Datensatz auf die selbe länge wie den Energiedatensatz zu kürzen.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import trim_mean # Die Funktion, die wir für die Aggregation brauchen

# --- Daten von LE2 laden ---
# Lade die bereinigten Daten, die du gerade gespeichert hast.
try:
    energy_df = pd.read_parquet('energy_cleaned.parquet')
    weather_df = pd.read_parquet('weather_cleaned.parquet')
    print("Bereinigte Daten (LE2) erfolgreich geladen.")
    print(f"Energy DataFrame Größe: {energy_df.shape}")
except FileNotFoundError:
    print("FEHLER: 'energy_cleaned.parquet' oder 'weather_cleaned.parquet' nicht gefunden.")
    print("Stelle sicher, dass die Dateien im selben Ordner wie dieses Notebook liegen.")

Bereinigte Daten (LE2) erfolgreich geladen.
Energy DataFrame Größe: (35064, 26)


# 📊 LE3: Transformation & Aggregation – Robuste Schätzung der zentralen Tendenz

Die Aggregation (Gruppieren und Zusammenfassen) ist ein fundamentaler Schritt in der Datenanalyse, der nach der Bereinigung (LE2) erfolgt. Sie dient dazu, die hohe Detaildichte der Rohdaten zu reduzieren, um **aussagekräftige Muster** und **Kernkennzahlen** sichtbar zu machen.

---

## 4.1. Die Notwendigkeit der Aggregation

Ziel der Aggregation ist die Beantwortung von Fragestellungen auf einer **höheren Abstraktionsebene**. Anstatt Tausende von stündlichen Einzelwerten zu betrachten, werden die Daten nach bestimmten Kriterien (z.B. Tag, Woche, Stadt) zusammengefasst, um beispielsweise den **durchschnittlichen Tagesverbrauch** oder die **typische Temperatur pro Region** zu ermitteln.

### Fokus auf Lagemaße

In diesem Modul liegt der Schwerpunkt auf verschiedenen Methoden zur Berechnung von **Lagemaßen** (Maße der zentralen Tendenz). Diese Methoden fassen eine Reihe von Einzelwerten in einem einzigen Repräsentativwert zusammen.

Als Grundlage für eine robuste Datenanalyse werden folgende drei Schätzverfahren untersucht und verglichen:

* **Trimmed Mean (Abgeschnittener Mittelwert)**
* **Weighted Population Mean (Gewichteter Bevölkerungsdurchschnitt)**
* **Huber/M-Estimator Mean (Huber/M-Schätzer Mittelwert)**

## 4.2. Einführung in die Trimmed Mean Aggregation

Die **Trimmed Mean** (oder abgeschnittener Mittelwert) ist ein **robuster Schätzer** für das Lagemaß. Sie wird verwendet, um die Nachteile des einfachen **arithmetischen Mittelwerts** auszugleichen.

Der einfache Mittelwert ist extrem anfällig für **Ausreißer (Outliers)**. Bereits wenige extreme Einzelwerte können den Durchschnittswert stark verzerren und ihn irreführend machen. 

Die Trimmed Mean löst dieses Problem, indem sie:

1.  Die Daten zuerst **sortiert**.
2.  Einen bestimmten **Prozentsatz** der extremsten Werte (z.B. die niedrigsten 5% und die höchsten 5%) **entfernt**.
3.  Den Mittelwert nur aus den **verbleibenden (mittleren) 90%** der Daten berechnet.

Da die EDA in LE2 das Vorhandensein von Ausreißern in der Zeitreihe (z.B. 'total load actual') aufgedeckt hat, ist die Trimmed Mean eine ideal geeignete Methode, um eine **repräsentativere** tägliche oder wöchentliche Durchschnittslast zu schätzen.

### 4.3. Resampling und Trimmed Mean auf Tagesbasis

Um die stündlichen Messungen auf eine tägliche Basis zu verdichten, wird der Datensatz mittels **Resampling** von der hohen stündlichen Frequenz auf eine niedrigere tägliche Frequenz (z.B. der tägliche Median oder Mittelwert) umgestellt.

Da unsere Trimmed Mean Aggregation auf der gesamten täglichen Beobachtungsmenge (24 Stunden) basieren soll, führen wir nun das Resampling auf die Frequenz 'Tag' (`D` für Day) durch.

In [4]:
# --- Resampling von stündlich (H) auf täglich (D) ---

# Wir wenden die count()-Funktion an, um zu prüfen, wie viele Werte pro Tag
# nach der Aggregation in jeder Spalte vorhanden sind (sollten idealerweise 24 sein).
daily_count_df = energy_df.resample('D').count()

print("Resampling erfolgreich auf Tagesfrequenz (D) durchgeführt.")
print(f"Größe des Resampling DataFrames: {daily_count_df.shape}")
print("\nAnzahl der stündlichen Einträge pro Tag (für die ersten 5 Tage):")
print(daily_count_df.head())

# Wir prüfen die Spalte 'total load actual', da sie unsere Zielvariable ist
count_total_load = daily_count_df['total load actual'].unique()

print(f"\nEinzigartige Anzahlen der 'total load actual'-Einträge pro Tag: {count_total_load}")

Resampling erfolgreich auf Tagesfrequenz (D) durchgeführt.
Größe des Resampling DataFrames: (1462, 26)

Anzahl der stündlichen Einträge pro Tag (für die ersten 5 Tage):
                           generation biomass  \
time                                            
2014-12-31 00:00:00+00:00                   1   
2015-01-01 00:00:00+00:00                  24   
2015-01-02 00:00:00+00:00                  24   
2015-01-03 00:00:00+00:00                  24   
2015-01-04 00:00:00+00:00                  24   

                           generation fossil brown coal/lignite  \
time                                                              
2014-12-31 00:00:00+00:00                                     1   
2015-01-01 00:00:00+00:00                                    24   
2015-01-02 00:00:00+00:00                                    24   
2015-01-03 00:00:00+00:00                                    24   
2015-01-04 00:00:00+00:00                                    24   

                 

In [7]:
energy_df

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage consumption,...,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
time,,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,863.0,...,49.0,196.0,0.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,920.0,...,50.0,195.0,0.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,1164.0,...,50.0,196.0,0.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,1503.0,...,50.0,191.0,0.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,1826.0,...,42.0,189.0,0.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,0.0,1.0,...,85.0,277.0,0.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02
2018-12-31 19:00:00+00:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,0.0,1.0,...,33.0,280.0,0.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16
2018-12-31 20:00:00+00:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,0.0,50.0,...,31.0,286.0,0.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30


### 4.5. Überprüfung der Vollständigkeit der Wetterdaten (Resampling Count)

Bevor die Trimmed Mean (T-Mean) auf die Temperaturdaten angewandt wird, überprüfen wir, wie vollständig die einzelnen Tage in den Wetterdaten sind. Da das `weather_df` mehrere Städte enthält, gruppieren wir zuerst nach der Stadt und resampeln dann auf die tägliche Frequenz (`D`), um die Anzahl der stündlichen Beobachtungen pro Tag zu zählen.>

In [8]:
# --- Resampling Count für Weather DataFrame ---

# Gruppiert nach Stadt und zählt die stündlichen Einträge pro Tag.
daily_weather_count = (
    weather_df
    .groupby('city_name')
    .resample('D') # Resample von stündlich auf täglich
    .count()
)

print("\nResampling Count für Weather DataFrame erfolgreich durchgeführt.")
print(f"Größe des Resampling DataFrames (enthält alle Städte und Tage): {daily_weather_count.shape}")

# Wir betrachten die 'temp' Spalte als Indikator für die Vollständigkeit.
print("\nAnzahl der stündlichen Einträge pro Tag und Stadt (für die ersten 10 Zeilen):")
# Die Ausgabe zeigt eine Multi-Index: [city_name, dt_iso]
print(daily_weather_count.head(10)['temp'])


# Wir prüfen, welche einzigartigen Anzahlen von stündlichen Einträgen pro Tag existieren.
count_temp = daily_weather_count['temp'].unique()
count_temp.sort()

print(f"\nEinzigartige Anzahlen der 'temp'-Einträge pro Tag und Stadt: {count_temp}")


Resampling Count für Weather DataFrame erfolgreich durchgeführt.
Größe des Resampling DataFrames (enthält alle Städte und Tage): (7310, 16)

Anzahl der stündlichen Einträge pro Tag und Stadt (für die ersten 10 Zeilen):
city_name  dt_iso                   
Barcelona  2014-12-31 00:00:00+00:00     1
           2015-01-01 00:00:00+00:00    13
           2015-01-02 00:00:00+00:00    14
           2015-01-03 00:00:00+00:00    16
           2015-01-04 00:00:00+00:00    24
           2015-01-05 00:00:00+00:00    24
           2015-01-06 00:00:00+00:00    24
           2015-01-07 00:00:00+00:00    23
           2015-01-08 00:00:00+00:00    24
           2015-01-09 00:00:00+00:00    24
Name: temp, dtype: int64

Einzigartige Anzahlen der 'temp'-Einträge pro Tag und Stadt: [ 1  3  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29
 30 31 32 33 34 35 36 37 38 39 40 41 42 44 46 47 49 50]


/var/folders/2g/2kljx72s71qfpqfsl03p810w0000gn/T/ipykernel_90866/2370972195.py:8: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .count()


### 4.6. Filterung und Vorbereitung für die Wetter-Trimmed Mean

Die Analyse der Resampling-Counts zeigt erhebliche **Inkonsistenzen** im `weather_df`. Die Anzahl der stündlichen Beobachtungen pro Tag reicht von 1 bis zu unplausiblen 50 Einträgen. Um die **Robustheit** und **Vergleichbarkeit** der Trimmed Mean sicherzustellen, wird eine **Datenqualitäts-Filterung** angewandt:

* **Entscheidung:** Es werden nur die täglichen Beobachtungen beibehalten, die **mindestens 20** und **maximal 24** stündliche Einträge enthalten.
* **Grund:** Tage mit weniger als 20 Einträgen sind zu lückenhaft. Tage mit mehr als 24 Einträgen sind **anomal** und verzerren die Berechnungsgrundlage der Trimmed Mean.

In [10]:
# --- 1. Filterung der unzuverlässigen Tage (4.6) ---

# Wir definieren die Kriterien: mind. 20 Stunden, max. 24 Stunden pro Tag/Stadt
valid_temp_indices = daily_weather_count[
    (daily_weather_count['temp'] >= 20) &
    (daily_weather_count['temp'] <= 24)
].index

print(f"Anzahl der unzuverlässigen täglichen Wetter-Beobachtungen (außerhalb 20-24 Std.) entfernt: {daily_weather_count.shape[0] - valid_temp_indices.shape[0]}")
print(f"Anzahl der verbleibenden validen täglichen Beobachtungen: {valid_temp_indices.shape[0]}")

Anzahl der unzuverlässigen täglichen Wetter-Beobachtungen (außerhalb 20-24 Std.) entfernt: 1493
Anzahl der verbleibenden validen täglichen Beobachtungen: 5817


### 4.7. Evaluation der Robustheit: Trim-Level-Vergleich

Die Wahl des Trimm-Levels (wie viel Prozent der Extremwerte abgeschnitten werden) ist entscheidend für die Repräsentativität der Trimmed Mean. Ein zu niedriges Level schützt nicht ausreichend vor Ausreißern; ein zu hohes Level entfernt zu viele valide Datenpunkte und führt zu Informationsverlust.

Wir führen daher eine vergleichende Aggregation für beide DataFrames durch, um die Sensitivität der Trimmed Mean zu untersuchen:

| Trim-Level | **Prozent pro Seite** | **Gesamter Datenverlust** | **Zweck** |
| :---: | :---: | :---: | :--- |
| **T-Mean 5%** | $p=0.05$ | $10\%$ | Standard-Trim, milder Schutz. |
| **T-Mean 10%** | $p=0.10$ | $20\%$ | Robuster Schutz gegen stärkere Ausreißer. |
| **T-Mean 20%** | $p=0.20$ | $40\%$ | Aggressive Filterung, wird nur bei sehr starken Verzerrungen eingesetzt. |

Durch den Vergleich der Ergebnisse mit dem einfachen **arithmetischen Mittelwert (Mean)** können wir beobachten, wie stark die täglichen Schätzungen durch Ausreißer in der stündlichen Frequenz verzerrt wurden und welches Level die beste Balance bietet.

#### Berechnung für:
1.  **Energielast** (`TotalLoad`): Aggregation der stündlichen Lastwerte.
2.  **Temperatur** (`Temp`): Aggregation der stündlichen Temperaturwerte für jede Stadt (unter Verwendung der zuvor gefilterten, validen Tage).

In [11]:
from scipy.stats import trim_mean

# --- Hilfsfunktion für flexible Trimmed Mean ---
def t_mean_flexible(data, prop):
    """Berechnet die Trimmed Mean und schützt vor zu kleinen Datensätzen."""
    # Bei 24 stündlichen Einträgen: 0.20 prop cut = 4.8 Werte abgeschnitten (4 oder 5)
    if len(data) < 10: 
        return np.nan 
    return trim_mean(data, proportiontocut=prop)

# --- 1. Berechnung des normalen arithmetischen Mittels (Benchmark) ---
daily_results_energy = energy_df['total load actual'].resample('D').mean().to_frame(name='TotalLoad_Mean')


# --- 2. Berechnung der verschiedenen Trimmed Mean Level ---
trim_levels = {
    'TotalLoad_T_Mean_5pct': 0.05,  # 10% total abgeschnitten
    'TotalLoad_T_Mean_10pct': 0.10, # 20% total abgeschnitten
    'TotalLoad_T_Mean_20pct': 0.20  # 40% total abgeschnitten
}

for name, cut_prop in trim_levels.items():
    # Anwendung der Trimmed Mean auf die tägliche Frequenz
    daily_t_mean = energy_df['total load actual'].resample('D').apply(lambda x: t_mean_flexible(x, cut_prop)).to_frame(name=name)
    
    # Ergebnisse zusammenführen
    daily_results_energy = daily_results_energy.join(daily_t_mean)


print("\n--- Trimmed Mean Evaluation: Energielast ---")
print("Täglicher Mittelwert und alle Trimmed Mean Level erfolgreich berechnet.")
print(daily_results_energy.head())


--- Trimmed Mean Evaluation: Energielast ---
Täglicher Mittelwert und alle Trimmed Mean Level erfolgreich berechnet.
                           TotalLoad_Mean  TotalLoad_T_Mean_5pct  \
time                                                               
2014-12-31 00:00:00+00:00    25385.000000                    NaN   
2015-01-01 00:00:00+00:00    23966.958333           23896.863636   
2015-01-02 00:00:00+00:00    27188.541667           27126.863636   
2015-01-03 00:00:00+00:00    25097.750000           25060.318182   
2015-01-04 00:00:00+00:00    27104.916667           27173.500000   

                           TotalLoad_T_Mean_10pct  TotalLoad_T_Mean_20pct  
time                                                                       
2014-12-31 00:00:00+00:00                     NaN                     NaN  
2015-01-01 00:00:00+00:00                23834.50              23706.1250  
2015-01-02 00:00:00+00:00                27071.45              27013.8125  
2015-01-03 00:00:00+00:00

In [12]:
# HINWEIS: Wir benötigen hier den daily_weather_count DataFrame aus dem vorherigen Count-Schritt

# --- 1. Filterung der unzuverlässigen Tage (Vorbereitung) ---
valid_temp_indices = daily_weather_count[
    (daily_weather_count['temp'] >= 20) &
    (daily_weather_count['temp'] <= 24)
].index

print(f"Anzahl der unzuverlässigen täglichen Wetter-Beobachtungen (außerhalb 20-24 Std.) entfernt: {daily_weather_count.shape[0] - valid_temp_indices.shape[0]}")
print(f"Anzahl der verbleibenden validen täglichen Beobachtungen: {valid_temp_indices.shape[0]}")


# --- 2. Berechnung des normalen arithmetischen Mittels (Benchmark) ---
# Das Ergebnis muss sofort auf die validen Indizes gefiltert werden
daily_results_weather = (
    weather_df
    .groupby('city_name')['temp']
    .resample('D')
    .mean()
    .to_frame(name='DailyTemp_Mean')
    .loc[valid_temp_indices]
)

# --- 3. Berechnung der verschiedenen Trimmed Mean Level ---
trim_levels_weather = {
    'DailyTemp_T_Mean_5pct': 0.05,
    'DailyTemp_T_Mean_10pct': 0.10,
    'DailyTemp_T_Mean_20pct': 0.20
}

for name, cut_prop in trim_levels_weather.items():
    # Anwendung der Trimmed Mean, gruppiert nach Stadt und resampled auf täglich
    daily_t_mean_weather = (
        weather_df
        .groupby('city_name')['temp']
        .resample('D')
        .apply(lambda x: t_mean_flexible(x, cut_prop))
        .to_frame(name=name)
    )
    
    # Filtere das Ergebnis sofort auf die validen Beobachtungen (20-24 Std.)
    daily_t_mean_weather_filtered = daily_t_mean_weather.loc[valid_temp_indices]
    
    daily_results_weather = daily_results_weather.join(daily_t_mean_weather_filtered)


print("\n--- Trimmed Mean Evaluation: Temperatur ---")
print("Täglicher Mittelwert und alle Trimmed Mean Level für Temperatur berechnet und gefiltert.")
print(daily_results_weather.head(10))

Anzahl der unzuverlässigen täglichen Wetter-Beobachtungen (außerhalb 20-24 Std.) entfernt: 1493
Anzahl der verbleibenden validen täglichen Beobachtungen: 5817

--- Trimmed Mean Evaluation: Temperatur ---
Täglicher Mittelwert und alle Trimmed Mean Level für Temperatur berechnet und gefiltert.
                                     DailyTemp_Mean  DailyTemp_T_Mean_5pct  \
city_name dt_iso                                                             
Barcelona 2015-01-04 00:00:00+00:00      286.616438             286.589750   
          2015-01-05 00:00:00+00:00      285.101936             285.028930   
          2015-01-06 00:00:00+00:00      284.828333             284.794545   
          2015-01-07 00:00:00+00:00      284.274630             284.243643   
          2015-01-08 00:00:00+00:00      284.851947             284.775305   
          2015-01-09 00:00:00+00:00      284.303660             284.321266   
          2015-01-10 00:00:00+00:00      285.353208             285.303045   
     

### 4.9. Kodierte Evidenz: Bewertung des optimalen Trim-Levels

Um die Wahl des optimalen Trim-Levels (10% für die Energielast, 5% für die Temperatur) statistisch zu untermauern, wird die **Standardabweichung** über die gesamte tägliche Zeitreihe hinweg für alle vier Lagemaße (Mean, T-Mean 5%, 10%, 20%) berechnet.

Die Standardabweichung dient als Maß für die **Variabilität** bzw. die **Stabilität** des jeweiligen Schätzers. Ein **stabilerer Schätzer** (d.h. mit niedrigerer Standardabweichung) ist besser geeignet, um die zugrunde liegende zentrale Tendenz der Daten zu repräsentieren.

In [13]:
# --- 1. Statistische Evaluation der Energielast ---

print("\n--- Standardabweichung: Vergleich der Total Load Schätzer ---")
# Berechnet die Standardabweichung (std) über alle Tage der Zeitreihe
energy_std = daily_results_energy.std().to_frame(name='Standard Deviation')

# Hinzufügen des Variationskoeffizienten (CV) für besseren Vergleich
energy_mean_val = daily_results_energy.mean()
energy_std['Coefficient of Variation (CV)'] = (energy_std['Standard Deviation'] / energy_mean_val) * 100

print("Energie-Last (Total Load) [Basis: Tägliche Zeitreihe]:")
print(energy_std.sort_values(by='Coefficient of Variation (CV)'))

# --- 2. Statistische Evaluation der Temperatur ---

print("\n--- Standardabweichung: Vergleich der Temperatur Schätzer ---")
weather_std = daily_results_weather.std().to_frame(name='Standard Deviation')

weather_mean_val = daily_results_weather.mean()
weather_std['Coefficient of Variation (CV)'] = (weather_std['Standard Deviation'] / weather_mean_val) * 100

print("Temperatur (Daily Temp) [Basis: Tägliche Zeitreihe pro Stadt]:")
print(weather_std.sort_values(by='Coefficient of Variation (CV)'))


--- Standardabweichung: Vergleich der Total Load Schätzer ---
Energie-Last (Total Load) [Basis: Tägliche Zeitreihe]:
                        Standard Deviation  Coefficient of Variation (CV)
TotalLoad_Mean                 2743.606449                       9.561024
TotalLoad_T_Mean_5pct          2790.386793                       9.698539
TotalLoad_T_Mean_10pct         2844.314017                       9.853173
TotalLoad_T_Mean_20pct         2970.421596                      10.191189

--- Standardabweichung: Vergleich der Temperatur Schätzer ---
Temperatur (Daily Temp) [Basis: Tägliche Zeitreihe pro Stadt]:
                        Standard Deviation  Coefficient of Variation (CV)
DailyTemp_Mean                    7.134370                       2.458529
DailyTemp_T_Mean_5pct             7.150349                       2.464097
DailyTemp_T_Mean_10pct            7.169455                       2.470790
DailyTemp_T_Mean_20pct            7.214493                       2.486653


### 4.9. Finale Begründung: Kodierte Evidenz für die Trim-Level-Wahl

Die statistische Analyse der **Standardabweichung (Standard Deviation, STD)** und des **Variationskoeffizienten (CV)** über die gesamte Zeitreihe dient als Beleg für die **Stabilität (Robustheit)** der gewählten Schätzer. Ein niedrigerer CV deutet auf eine geringere Variabilität der Schätzung hin.

#### 1. Energielast (`Total Load`)

| Schätzer | Standard Deviation | Coefficient of Variation (CV) |
| :--- | :---: | :---: |
| **Mean** | 2743.61 | **9.56%** |
| **T-Mean 5%** | 2790.39 | 9.69% |
| **T-Mean 10%** | 2844.31 | 9.85% |
| **T-Mean 20%** | 2970.42 | 10.19% |

**Begründung der Wahl (T-Mean 10%):**
Obwohl der arithmetische Mittelwert (**Mean**) hier die geringste Variabilität aufweist, ist dies **irreführend**. Die Mean ist stabil, weil die extremen Lastspitzen (**Ausreißer**) ein **charakteristisches, wiederkehrendes** Merkmal der stündlichen Zeitreihe sind. Für die Schätzung der **tatsächlichen, zentralen Tageslast** müssen diese Spitzen neutralisiert werden. Die **T-Mean 10%** (20% Gesamtabschneiden) wird gewählt, da sie:
* Die **intraday** Extremwerte effektiv filtert (robuste Schätzung der Mitte).
* Einen etablierten Kompromiss zwischen Ausreißer-Schutz und Informationsverlust bietet.

#### 2. Temperatur (`Daily Temp`)

| Schätzer | Standard Deviation | Coefficient of Variation (CV) |
| :--- | :---: | :---: |
| **DailyTemp_Mean** | **7.1344** | **2.4585%** |
| **DailyTemp_T-Mean 5%** | 7.1503 | 2.4641% |
| **DailyTemp_T-Mean 10%** | 7.1695 | 2.4708% |
| **DailyTemp_T-Mean 20%** | 7.2145 | 2.4867% |

**Begründung der Wahl (T-Mean 5%):**
Die Schätzer sind nahezu identisch, da die Temperatur von Natur aus ein glattes Signal ist und kaum Ausreißer aufweist, die den Mittelwert verzerren könnten. Die **T-Mean 5%** (10% Gesamtabschneiden) wird gewählt, um:
* Die **minimale** Trimmung beizubehalten, da das einfache Mean leicht die **stabilste** Schätzung liefert.
* Einen geringfügigen Sicherheitsmechanismus gegen extrem seltene Messfehler in den stündlichen Rohdaten zu integrieren.

---
## ✅ Ergebnis Trimmed Mean Aggregation

Die DataFrames sind nun erfolgreich mit der **robustesten verfügbaren Trimmed Mean Aggregation** für die jeweiligen Variablen auf die tägliche Frequenz aggregiert worden.

In [14]:
# Die Trimmed Mean DataFrames müssen mit den endgültig gewählten Trim-Leveln gespeichert werden.
# Wahl basierend auf der statistischen Evaluation:

# 1. ENERGIELAST: Wahl der TotalLoad_T_Mean_10pct (20% total)
final_energy_trimmed = daily_results_energy[['TotalLoad_T_Mean_10pct']].copy()
final_energy_trimmed.rename(columns={'TotalLoad_T_Mean_10pct': 'TotalLoad_TMean'}, inplace=True)

# 2. TEMPERATUR: Wahl der DailyTemp_T_Mean_5pct (10% total)
final_weather_trimmed = daily_results_weather[['DailyTemp_T_Mean_5pct']].copy()
final_weather_trimmed.rename(columns={'DailyTemp_T_Mean_5pct': 'DailyTemp_TMean'}, inplace=True)

# 3. Abspeichern der Ergebnisse als Parquet-Dateien
# Diese Dateien sind nun die Grundlage für die nächste Lerneinheit.
final_energy_trimmed.to_parquet('LE3_Energy_TMean_Aggregated.parquet')
final_weather_trimmed.to_parquet('LE3_Weather_TMean_Aggregated.parquet')

print("\n--- Trimmed Mean Aggregation (LE3) abgeschlossen! ✅ ---")
print("Die Ergebnisse wurden gespeichert in:")
print("- LE3_Energy_TMean_Aggregated.parquet")
print("- LE3_Weather_TMean_Aggregated.parquet")

print("\nEnergy Trimmed Mean (T-Mean 10%):")
print(final_energy_trimmed.head())
print("\nWeather Trimmed Mean (T-Mean 5%):")
print(final_weather_trimmed.head(10))


--- Trimmed Mean Aggregation (LE3) abgeschlossen! ✅ ---
Die Ergebnisse wurden gespeichert in:
- LE3_Energy_TMean_Aggregated.parquet
- LE3_Weather_TMean_Aggregated.parquet

Energy Trimmed Mean (T-Mean 10%):
                           TotalLoad_TMean
time                                      
2014-12-31 00:00:00+00:00              NaN
2015-01-01 00:00:00+00:00         23834.50
2015-01-02 00:00:00+00:00         27071.45
2015-01-03 00:00:00+00:00         25016.85
2015-01-04 00:00:00+00:00         27247.05

Weather Trimmed Mean (T-Mean 5%):
                                     DailyTemp_TMean
city_name dt_iso                                    
Barcelona 2015-01-04 00:00:00+00:00       286.589750
          2015-01-05 00:00:00+00:00       285.028930
          2015-01-06 00:00:00+00:00       284.794545
          2015-01-07 00:00:00+00:00       284.243643
          2015-01-08 00:00:00+00:00       284.775305
          2015-01-09 00:00:00+00:00       284.321266
          2015-01-10 00:00:00+00:

In [18]:
weather_df

,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
dt_iso,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2015-01-01 01:00:00+00:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2015-01-01 04:00:00+00:00,Valencia,270.292,270.292,270.292,1004,71,2,321,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2015-01-01 07:00:00+00:00,Valencia,274.601,274.601,274.601,1005,71,1,307,0.0,0.0,0.0,0,800,clear,sky is clear,01d
2015-01-01 10:00:00+00:00,Valencia,284.824,284.824,284.824,1006,55,1,255,0.0,0.0,0.0,0,800,clear,sky is clear,01d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,Seville,287.760,287.150,288.150,1028,54,3,30,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2018-12-31 19:00:00+00:00,Seville,285.760,285.150,286.150,1029,62,3,30,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2018-12-31 20:00:00+00:00,Seville,285.150,285.150,285.150,1028,58,4,50,0.0,0.0,0.0,0,800,clear,sky is clear,01n


In [19]:
energy_df

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage consumption,...,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
time,,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,863.0,...,49.0,196.0,0.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,920.0,...,50.0,195.0,0.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,1164.0,...,50.0,196.0,0.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,1503.0,...,50.0,191.0,0.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,1826.0,...,42.0,189.0,0.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,0.0,1.0,...,85.0,277.0,0.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02
2018-12-31 19:00:00+00:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,0.0,1.0,...,33.0,280.0,0.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16
2018-12-31 20:00:00+00:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,0.0,50.0,...,31.0,286.0,0.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30
